In [8]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf
import numpy as np

In [2]:
with tf.device('/cpu:0'):
    input_tensor = Input((299, 299, 3))
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1)(p_out)
    predictions = Dense(2, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.load_weights('Xception_finetune_10.h5')

parallel_model = multi_gpu_model(model, gpus=2)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [3]:
batch_size = 128
img_size = (299, 299)

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("/usr/super-speed-data/train-data-xception-pre/20181023/test-ready", 
                                         img_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
all_test_results = parallel_model.predict_generator(test_generator, (test_img_nums//batch_size) + 1, verbose=1)
all_test_results = all_test_results[:test_img_nums]
all_labels = test_generator.classes

Found 95694 images belonging to 2 classes.
748/748 [==============================] - 314s 419ms/step


In [12]:
class_label_dict = test_generator.class_indices
print(class_label_dict)

# should be 1(normal) but predict is 0(abnormal)
# 假陽性
error1 = []
# should be 0(abnormal) but predict is 1(normal)
# 假陰性
error2 = []

for i, test_result in enumerate(all_test_results):
    if (np.argmax(test_result) == 0 and all_labels[i] == 1):
        error1.append(i)
    if (np.argmax(test_result) == 1 and all_labels[i] == 0):
        error2.append(i)

{'normal': 1, 'abnormal': 0}


In [13]:
print(error1)
print(error2)

[48547, 48548, 48549, 48550, 48551, 48552, 48553, 48554, 48555, 48556, 48557, 48959, 48960, 48961, 48962, 48963, 48964, 48965, 48966, 48967, 48968, 48969, 51024, 52837, 53620, 53826, 53976, 54093, 55242, 55439, 55442, 56002, 56006, 58314, 60019, 62937, 64577, 64578, 64580, 67448, 68382, 69110, 69195, 73915, 77633, 78179, 78180, 78181, 78182, 78183, 78184, 78185, 78186, 78187, 78188, 78189, 78190, 78557, 78701, 78706, 78996, 80489, 80491, 80513, 80515, 80527, 80555, 80556, 80557, 80558, 80559, 80560, 80561, 80562, 80563, 80564, 80565, 80566, 80567, 80647, 80656, 80657, 80659, 80661, 80665, 80751, 80769, 81073, 81233, 81236, 81238, 81522, 81819, 82933, 83117, 83314, 83317, 83318, 83319, 83330, 83331, 83333, 83505, 83559, 83788, 83790, 83909, 84243, 87947, 88710, 88713, 89054, 89373, 89387, 89755, 89759, 89762, 89782, 89791, 89802, 89804, 89920, 90399, 90400, 90454, 92169, 92404, 92409, 92476, 93873, 94831]
[345, 674, 680, 688, 690, 693, 724, 745, 752, 1250, 1732, 1808, 1815, 2598, 2673, 

In [35]:
import os
import shutil

all_test_files_dir = "/usr/super-speed-data/train-data-xception-pre/20181023/test-ready/"
error1_dir = "/usr/super-speed-data/train-data-xception-pre/20181023/test_debug/error1/"
error2_dir = "/usr/super-speed-data/train-data-xception-pre/20181023/test_debug/error2/"

all_files = []

sub_dirs = os.listdir(all_test_files_dir)
sub_dirs.sort()
for sub_dir in sub_dirs:
    files = os.listdir(all_test_files_dir + sub_dir)
    files.sort()
    all_files += files

for index in error1:
    shutil.copy(all_test_files_dir + "normal/" + all_files[index], error1_dir)

for index in error2:
    shutil.copy(all_test_files_dir + "abnormal/" + all_files[index], error2_dir)

In [40]:
print(len(all_files))

95694


In [41]:
print(all_files[47000:48001])

['2018-06-01_12_40_20_50069_34745.jpg', '2018-06-01_12_40_20_50109_27167.jpg', '2018-06-01_12_40_20_50162_24919.jpg', '2018-06-01_12_40_20_50202_36546.jpg', '2018-06-01_12_40_20_50228_30183.jpg', '2018-06-01_12_40_20_50232_30013.jpg', '2018-06-01_12_40_20_50267_25557.jpg', '2018-06-01_12_40_20_50316_28421.jpg', '2018-06-01_12_40_20_50409_29204.jpg', '2018-06-01_12_40_20_50477_35323.jpg', '2018-06-01_12_40_20_50506_34864.jpg', '2018-06-01_12_40_20_50506_35270.jpg', '2018-06-01_12_40_20_50528_28909.jpg', '2018-06-01_12_40_20_50548_29816.jpg', '2018-06-01_12_40_20_50549_35362.jpg', '2018-06-01_12_40_20_5058_27523.jpg', '2018-06-01_12_40_20_50598_29728.jpg', '2018-06-01_12_40_20_50606_34832.jpg', '2018-06-01_12_40_20_50731_25279.jpg', '2018-06-01_12_40_20_50732_33039.jpg', '2018-06-01_12_40_20_50827_34782.jpg', '2018-06-01_12_40_20_51051_25183.jpg', '2018-06-01_12_40_20_51200_32337.jpg', '2018-06-01_12_40_20_5120_32539.jpg', '2018-06-01_12_40_20_51225_28646.jpg', '2018-06-01_12_40_20_51282

In [42]:
print(all_files[95690:95694])

['TC18055267_x37400_y23378_w1216_h1216_s28.jpg', 'TC18055267_x42000_y44870_w1216_h1216_s61.jpg', 'TC18055267_x46677_y37917_w1216_h1216_s181.jpg', 'TC18055267_x47029_y37048_w1216_h1216_s1.jpg']
